In [ ]:
import pandas as pd
import os
import re
import glob


data_dir = 'BACI_HS92_V202501'

country_codes_file = os.path.join(data_dir, 'country_codes_V202501.csv')
country_codes_df = pd.read_csv(country_codes_file)

country_mapping = dict(zip(country_codes_df['country_code'], country_codes_df['country_name']))

ukraine_code = country_codes_df[country_codes_df['country_name'] == 'Ukraine']['country_code'].values[0]
print(f"Ukraine's country code: {ukraine_code}")

wheat_codes = [
    100110,  # Durum wheat
    100190,  # Other wheat and meslin (older code)
    100199,  # Other wheat and meslin (newer code)
    110100,  # Wheat or meslin flour
    110311,  # Cereal groats, meal of wheat
    110321,  # Cereal grains otherwise worked: of wheat
    110811,  # Wheat starch
    110900   # Wheat gluten
]

maize_codes = [
    100510,  # Maize (corn): seed
    100590,  # Maize (corn): other than seed
    110220,  # Maize (corn) flour
    110313,  # Cereal groats, meal of maize (corn)
    110323,  # Cereal grains otherwise worked: of maize (corn)
    110812   # Maize (corn) starch
]

# Create a dictionary for product codes
product_mapping = {}
for code in wheat_codes:
    product_mapping[code] = 'Wheat'
for code in maize_codes:
    product_mapping[code] = 'Maize'

# Load product codes file to get descriptions
product_codes_file = os.path.join(data_dir, 'product_codes_HS92_V202501.csv')
try:
    product_df = pd.read_csv(product_codes_file)
    # Create a mapping from product code to description
    product_desc_mapping = dict(zip(
        [int(code) for code in product_df.iloc[:, 0]], 
        product_df.iloc[:, 1]
    ))
    # Add descriptions to our codes for verification
    print("Wheat products in our analysis:")
    for code in wheat_codes:
        if code in product_desc_mapping:
            print(f"{code}: {product_desc_mapping[code]}")
    print("\nMaize products in our analysis:")
    for code in maize_codes:
        if code in product_desc_mapping:
            print(f"{code}: {product_desc_mapping[code]}")
except:
    print("Product codes file not found or could not be processed. Continuing without descriptions.")

# Find all BACI data files in the directory
file_pattern = os.path.join(data_dir, 'BACI_HS92_Y*_V202501.csv')
baci_files = glob.glob(file_pattern)

# Sort files by year
def extract_year(filename):
    match = re.search(r'Y(\d{4})_', filename)
    if match:
        return int(match.group(1))
    return 0

baci_files.sort(key=extract_year)

print(f"Found {len(baci_files)} BACI data files from {extract_year(baci_files[0])} to {extract_year(baci_files[-1])}")

# Function to process a single BACI file
def process_baci_file(file_path):
    # Extract year from filename
    year_match = re.search(r'Y(\d{4})_', file_path)
    year = year_match.group(1) if year_match else "Unknown"
    
    print(f"Processing data for year {year}...")
    
    try:
        # Read the file
        data = pd.read_csv(file_path)
        print(f"  Loaded file with {len(data):,} rows")
        
        # Filter for Ukraine exports and wheat/maize products
        ukraine_exports = data[
            (data['i'] == ukraine_code) & 
            (data['k'].isin(wheat_codes + maize_codes))
        ]
        
        if ukraine_exports.empty:
            print(f"  No Ukraine wheat/maize exports found for {year}")
            return None
        
        print(f"  Found {len(ukraine_exports):,} rows of Ukraine wheat/maize exports")
        
        # Rename columns to be descriptive
        ukraine_exports = ukraine_exports.rename(columns={
            't': 'year',
            'i': 'exporter_code',
            'j': 'importer_code',
            'k': 'product_code',
            'v': 'value_thousands_usd',
            'q': 'quantity'
        })
        
        # If hs4 column exists, rename it
        if 'hs4' in ukraine_exports.columns:
            ukraine_exports = ukraine_exports.rename(columns={'hs4': 'product_category_code'})
        
        # Add human-readable columns
        ukraine_exports['exporter'] = 'Ukraine'
        ukraine_exports['importer'] = ukraine_exports['importer_code'].map(country_mapping).fillna('Unknown')
        ukraine_exports['product'] = ukraine_exports['product_code'].map(product_mapping)
        
        # Add value in millions USD for convenience
        ukraine_exports['value_million_usd'] = ukraine_exports['value_thousands_usd'] / 1000
        
        return ukraine_exports
    
    except Exception as e:
        print(f"  Error processing file {file_path}: {e}")
        return None

# Process all BACI files
all_data = []
for file in baci_files:
    year_data = process_baci_file(file)
    if year_data is not None:
        all_data.append(year_data)

if all_data:
    # Combine all years into one DataFrame
    combined_data = pd.concat(all_data, ignore_index=True)
    print(f"\nCombined data has {len(combined_data):,} rows spanning from {combined_data['year'].min()} to {combined_data['year'].max()}")
    
    # Save to CSV with descriptive column names
    output_file = 'data/ukraine_wheat_maize_exports_1995_2023.csv'
    combined_data.to_csv(output_file, index=False)
    print(f"\nSaved processed data to '{output_file}'")
    
    # Display column information
    print("\nColumns in the output file:")
    for col in combined_data.columns:
        print(f"- {col}")
    
    # Show a quick summary by year and product
    yearly_summary = combined_data.groupby(['year', 'product'])['value_million_usd'].sum().reset_index()
    print("\nYearly exports summary (million USD):")
    display(yearly_summary)
else:
    print("No data was processed. Check file paths and data.")


Ukraine's country code: 804
Product codes file not found or could not be processed. Continuing without descriptions.
Found 29 BACI data files from 1995 to 2023
Processing data for year 1995...
  Loaded file with 4,981,382 rows
  Found 31 rows of Ukraine wheat/maize exports
Processing data for year 1996...
  Loaded file with 5,337,010 rows
  Found 122 rows of Ukraine wheat/maize exports
Processing data for year 1997...
  Loaded file with 5,616,781 rows
  Found 103 rows of Ukraine wheat/maize exports
Processing data for year 1998...
  Loaded file with 5,847,753 rows
  Found 143 rows of Ukraine wheat/maize exports
Processing data for year 1999...
  Loaded file with 5,994,574 rows
  Found 161 rows of Ukraine wheat/maize exports
Processing data for year 2000...
  Loaded file with 7,272,711 rows
  Found 104 rows of Ukraine wheat/maize exports
Processing data for year 2001...
  Loaded file with 7,517,477 rows
  Found 125 rows of Ukraine wheat/maize exports
Processing data for year 2002...
  L

,year,product,value_million_usd
0,1995,Maize,2.325730
1,1995,Wheat,7.022525
2,1996,Maize,33.275195
3,1996,Wheat,391.489724
4,1997,Maize,15.361302
5,1997,Wheat,130.712965
6,1998,Maize,57.406580
7,1998,Wheat,346.660718
8,1999,Maize,30.944849
9,1999,Wheat,486.265787
